## This class will hold the database creation and population work

#### Install sqlalchemy_utils and Faker if they aren't already installed:

In [1]:
import importlib
import subprocess
import sys

def install_package(package_name: str):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])

packages = ['sqlalchemy_utils', 'Faker']

for package in packages:
    try:
        importlib.import_module(package)
        print(f"{package} is already installed.")
    except ModuleNotFoundError:
        print(f"{package} not found. Installing...")
        install_package(package)
        print(f"{package} has been installed.")


sqlalchemy_utils is already installed.
Faker not found. Installing...
Faker has been installed.


#### Create the database:

In [2]:
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine, text
from sqlalchemy_utils import database_exists, create_database

engine = create_engine('sqlite:///it_ticketing_system.db', echo=True)
if not database_exists(engine.url):
    create_database(engine.url)

if sqlalchemy.__version__ < '2.0.0':
    raise ValueError('Please upgrade your version of SQLAlchemy to 2.0.0 or greater')


### Using sqlite3 to drop tables if they're already created.

In [3]:
import sqlite3
db_file = 'it_ticketing_system.db'
cnn = sqlite3.connect(db_file)
cur = cnn.cursor()

table_names = []
results = cur.execute("SELECT tbl_name FROM sqlite_master")
for table_name in results:
    table_names.append(table_name[0])

for table_name in table_names:
    cur.execute(f'DROP TABLE IF EXISTS {table_name}')
    print(f'Dropped table \'{table_name}\'')

cnn.close()

Dropped table 'dim_organizations'
Dropped table 'dim_departments'
Dropped table 'dim_users'
Dropped table 'dim_technicians'
Dropped table 'fact_tickets'
Dropped table 'fact_ticket_lines'


#### Create the tables for the database:

In [ ]:
from sqlalchemy import Column, String, DateTime, Integer, ForeignKey
from sqlalchemy.orm import Session, DeclarativeBase, relationship
from sqlalchemy import select

class Base(DeclarativeBase):
    pass

class Organization(Base):
    __tablename__ = 'dim_organizations'
    
    organization_id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String, nullable=False)
    phone_number = Column(String)
    email_address = Column(String, nullable=False)
    state = Column(String, nullable=False)
    city = Column(String, nullable=False)
    zip_code = Column(Integer, nullable=False)
    street_address = Column(String, nullable=False)
    
    # Relationships
    users = relationship('User', back_populates='organization')
    departments = relationship('Department', back_populates='organization')

    def __repr__(self):
        return f'Organization(id={self.organization_id}, Name={self.name}, city={self.city}, state={self.state})'


class Department(Base):
    __tablename__ = 'dim_departments'
    
    department_id = Column(Integer, primary_key=True, autoincrement=True)
    organization_id = Column(Integer, ForeignKey('dim_organizations.organization_id'))
    name = Column(String, nullable=False)
    phone_number = Column(String)
    email_address = Column(String, nullable=False)
    
    # Relationships
    users = relationship('User', back_populates='department')
    organization = relationship('Organization', back_populates='departments')

    def __repr__(self):
        return f'Department(id={self.department_id}, Name={self.name}, Organization={self.organization.name})'


class User(Base):
    __tablename__ = 'dim_users'
    
    user_id = Column(Integer, primary_key=True, autoincrement=True)
    organization_id = Column(Integer, ForeignKey('dim_organizations.organization_id'))
    department_id = Column(Integer, ForeignKey('dim_departments.department_id'))
    last_name = Column(String, nullable=False)
    first_name = Column(String, nullable=False)
    phone_number = Column(String)
    email_address = Column(String, nullable=False)
    title = Column(String)
    
    # Relationships
    tickets = relationship('Ticket', back_populates='user')
    ticket_lines = relationship('TicketLine', back_populates='technician')
    technician_details = relationship('Technician', uselist=False, back_populates='user')
    organization = relationship('Organization', back_populates='users')
    department = relationship('Department', back_populates='users')


class Technician(Base):
    __tablename__ = 'dim_technicians'
    
    technician_id = Column(Integer, primary_key=True, autoincrement=True)
    user_id = Column(Integer, ForeignKey('dim_users.user_id'))
    manager_id = Column(Integer)
    
    # Relationships
    user = relationship('User', back_populates='technician_details')


class Ticket(Base):
    __tablename__ = 'fact_tickets'
    
    ticket_id = Column(Integer, primary_key=True, autoincrement=True)
    user_id = Column(Integer, ForeignKey('dim_users.user_id'))
    department_id = Column(Integer, ForeignKey('dim_departments.department_id'))
    prior_ticket_id = Column(Integer)
    ticket_category = Column(String, nullable=False)
    open_date_time = Column(DateTime, nullable=False)
    close_date_time = Column(DateTime)
    status = Column(String)
    description = Column(String, nullable=False)
    subject = Column(String, nullable=False)
    
    # Relationships
    user = relationship('User', back_populates='tickets')
    ticket_lines = relationship('TicketLine', back_populates='ticket')


class TicketLine(Base):
    __tablename__ = 'fact_ticket_lines'
    
    ticket_line_id = Column(Integer, primary_key=True, autoincrement=True)
    ticket_id = Column(Integer, ForeignKey('fact_tickets.ticket_id'))
    technician_id = Column(Integer, ForeignKey('dim_users.user_id'))
    assignment_date_time = Column(DateTime, nullable=False)
    completion_date_time = Column(DateTime)
    notes = Column(String)
    
    # Relationships
    technician = relationship('User', back_populates='ticket_lines')
    ticket = relationship('Ticket', back_populates='ticket_lines')

Base.metadata.create_all(engine)


#### Initial Database Definitions:

In [ ]:
db_file = 'it_ticketing_system.db'
cnn = sqlite3.connect(db_file)
cur = cnn.cursor()
tables = cur.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

def bool_to_yes_no(value):
    if value == 0:
        return 'NO'
    elif value == 1:
        return 'YES'

for table in tables:
    print('------------------')
    print('Table Name: ' + table[0])
    print('------------------')
    columns = cur.execute(f'PRAGMA table_info(\'%s\');' % table[0]).fetchall()
    for column in columns:
        col_id, col_name, col_type, col_notnull, col_default, col_pk = column
        print(f"  Column: {col_name}")
        print(f"    Type: {col_type}")
        print(f"    Not Null: {bool_to_yes_no(col_notnull)}")
        print(f"    Default Value: {col_default}")
        print(f"    Primary Key: {bool_to_yes_no(col_pk)}")

cnn.close()

#### Creating method to add be able to pass a List[Object] and have them added to the database, where Object is the table in ORM form

In [6]:
from sqlalchemy.orm import Session
from sqlalchemy.exc import SQLAlchemyError

def add_objects_to_db(session: Session, objects: list):
    try:
        session.add_all(objects)
        session.commit()
        return True
    except SQLAlchemyError as e:
        print(f"An error occurred: {e}")
        session.rollback()
        return False


#### Seed the tables:

In [7]:
from sqlalchemy.orm import Session
from faker import Faker
import random

fake = Faker()

def seed_organizations(session: Session, num_organizations: int):
    organizations = []
    for _ in range(num_organizations):
        organization = Organization(
            name=fake.company(),
            phone_number=fake.phone_number(),
            email_address=fake.company_email(),
            state=fake.state_abbr(),
            city=fake.city(),
            zip_code=fake.zipcode(),
            street_address=fake.street_address(),
        )
        organizations.append(organization)
    return add_objects_to_db(session, organizations)

def seed_departments(session: Session, num_departments_per_organization: int, organizations: list):
    departments = []
    for organization in organizations:
        for _ in range(num_departments_per_organization):
            department = Department(
                organization_id=organization.organization_id,
                name=fake.bs(),
                phone_number=fake.phone_number(),
                email_address=fake.company_email(),
            )
            departments.append(department)
    return add_objects_to_db(session, departments)

def seed_users(session: Session, num_users_per_department: int, departments: list):
    users = []
    for department in departments:
        for _ in range(num_users_per_department):
            user = User(
                organization_id=department.organization_id,
                department_id=department.department_id,
                last_name=fake.last_name(),
                first_name=fake.first_name(),
                phone_number=fake.phone_number(),
                email_address=fake.email(),
                title=fake.job(),
            )
            users.append(user)
    return add_objects_to_db(session, users)

def seed_tickets(session: Session, num_tickets_per_user: int, users: list):
    tickets = []
    for user in users:
        for _ in range(num_tickets_per_user):
            ticket = Ticket(
                user_id=user.user_id,
                department_id=user.department_id,
                ticket_category=fake.word(),
                open_date_time=fake.date_time_this_decade(),
                close_date_time=fake.date_time_this_decade(),
                status=fake.word(),
                description=fake.text(),
                subject=fake.sentence(),
            )
            tickets.append(ticket)
    return add_objects_to_db(session, tickets)

def seed_technicians(session: Session, users: list, num_technicians: int):
    technicians = []
    selected_users = random.sample(users, k=min(num_technicians, len(users)))

    for user in selected_users:
        technician = Technician(user_id=user.user_id)
        technicians.append(technician)

    return add_objects_to_db(session, technicians)

def seed_ticket_lines(session: Session, num_ticket_lines_per_ticket: int, tickets: list, technicians: list):
    ticket_lines = []
    for ticket in tickets:
        for _ in range(num_ticket_lines_per_ticket):
            technician = random.choice(technicians)

            ticket_line = TicketLine(
                ticket_id=ticket.ticket_id,
                technician_id=technician.technician_id,
                assignment_date_time=fake.date_time_this_decade(),
                completion_date_time=fake.date_time_this_decade(),
                notes=fake.text(),
            )
            ticket_lines.append(ticket_line)
    return add_objects_to_db(session, ticket_lines)

session = Session(engine)

#Create the organizations:
num_organizations = 5
seed_organizations(session, num_organizations)
organizations = session.query(Organization).all()

#Create the departments:
num_departments_per_organization = 3
seed_departments(session, num_departments_per_organization, organizations)
departments = session.query(Department).all()

#Create the users:
num_users_per_department = 10
seed_users(session, 10, departments)
users = session.query(User).all()

#Create the tickets:
num_tickets_per_user = 5
seed_tickets(session, num_tickets_per_user, users)
tickets = session.query(Ticket).all()

#Create the technicians:
num_technicians = 3
seed_technicians(session, users, num_technicians)
technicians = session.query(Technician).all()

#Create the ticket lines:
num_ticket_lines_per_ticket = 3
seed_ticket_lines(session, num_ticket_lines_per_ticket, tickets, technicians)
ticketlines = session.query(TicketLine).all()




2023-09-30 21:09:37,166 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-30 21:09:37,168 INFO sqlalchemy.engine.Engine INSERT INTO dim_organizations (name, phone_number, email_address, state, city, zip_code, street_address) VALUES (?, ?, ?, ?, ?, ?, ?) RETURNING organization_id
2023-09-30 21:09:37,169 INFO sqlalchemy.engine.Engine [generated in 0.00016s (insertmanyvalues) 1/5 (ordered; batch not supported)] ('Jennings-Gonzalez', '556-446-8262', 'scott17@decker.com', 'WA', 'Port Michaeltown', '27915', '2102 Kyle Ranch')
2023-09-30 21:09:37,177 INFO sqlalchemy.engine.Engine INSERT INTO dim_organizations (name, phone_number, email_address, state, city, zip_code, street_address) VALUES (?, ?, ?, ?, ?, ?, ?) RETURNING organization_id
2023-09-30 21:09:37,178 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/5 (ordered; batch not supported)] ('Mckee and Sons', '001-279-281-4374x8917', 'tapiacarl@collins.biz', 'WY', 'Lutzborough', '38014', '889 Marshall Run Suite 769')
2023-09-30 21:09:3

In [8]:


print(f'date_time_this_decade: {fake.date_time_this_decade()}')
print(f'word: {fake.word()}')
print(f'sentence: {fake.sentence()}')
print(f'text: {fake.text()}')
print(f'first_name: {fake.first_name()}')
print(f'last_name: {fake.last_name()}')
print(f'phone_number: {fake.phone_number()}')
print(f'email: {fake.email()}')
print(f'job: {fake.job()}')
print(f'company: {fake.company()}')
print(f'company_email: {fake.company_email()}')
print(f'state_abbr: {fake.state_abbr()}')
print(f'city: {fake.city()}')
print(f'zipcode: {fake.zipcode()}')
print(f'street_address: {fake.street_address()}')
print(f'bs: {fake.bs()}')





#Tickets:
#ticket_category
#open_date_time
#close_date_time
#status
#description
#subject
#
#Ticket Lines:
#assignment_date_time
#completion_date_time
#notes
#
#Users
#last_name
#first_name
#phone_number
#email_address
#title
#
#Departments
#name
#phone_number
#
#Organizations
#name
#phone_number
#email_address
#state
#city
#zip_code
#street_address
#
#
#






date_time_this_decade: 2021-10-11 15:42:55
word: like
sentence: They task audience trouble power performance major.
text: Vote create population air. Treat range large after see public race.
Just force effect it and now letter either. Others writer thousand piece single already.
first_name: Jose
last_name: Morse
phone_number: +1-676-898-1361
email: owilson@example.org
job: Bookseller
company: David-Stephens
company_email: uyoung@richardson.info
state_abbr: SD
city: Jonathanland
zipcode: 39121
street_address: 597 Cynthia Villages
bs: empower next-generation vortals


In [12]:
import pandas as pd

date_time_pre=fake.date_time_this_decade()

hours_taken = fake.random_int(min=0, max=4)
minutes_taken = fake.random_int(min=0, max=59)

date_time_post = date_time_pre + pd.DateOffset(hours=hours_taken, minutes=minutes_taken)


print(date_time_pre)
print(hours_taken)

print(minutes_taken)
print(date_time_post)


2022-08-07 09:54:28
1
23
2022-08-07 11:17:28


In [24]:
department_names = [
    'Department 1'
    , 'Department 2'
    , 'Department 3'
    , 'Department 4'
    , 'Department 5'
]

departments = session.query(Department).all()

for department in departments:
    selected_department = fake.random_int(min=0, max=len(department_names)-1)
    department.name = department_names[selected_department]
session.commit()

#123456789


2023-09-30 21:55:46,563 INFO sqlalchemy.engine.Engine UPDATE dim_departments SET name=? WHERE dim_departments.department_id = ?
2023-09-30 21:55:46,565 INFO sqlalchemy.engine.Engine [cached since 436.7s ago] [('Department 2', 3), ('Department 1', 5), ('Department 2', 6), ('Department 1', 7), ('Department 3', 8), ('Department 2', 9), ('Department 5', 10)]
2023-09-30 21:55:46,572 INFO sqlalchemy.engine.Engine SELECT dim_departments.department_id AS dim_departments_department_id, dim_departments.organization_id AS dim_departments_organization_id, dim_departments.name AS dim_departments_name, dim_departments.phone_number AS dim_departments_phone_number, dim_departments.email_address AS dim_departments_email_address 
FROM dim_departments
2023-09-30 21:55:46,574 INFO sqlalchemy.engine.Engine [cached since 2769s ago] ()
2023-09-30 21:55:46,577 INFO sqlalchemy.engine.Engine UPDATE dim_departments SET name=? WHERE dim_departments.department_id = ?
2023-09-30 21:55:46,580 INFO sqlalchemy.engine.